In [10]:
from __future__ import print_function
import csv
import pickle
import numpy as np
from PIL import Image
import time
from keras.applications import*
import keras
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, Flatten
from keras.layers import *
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.utils import multi_gpu_model
import pickle
from utils import *

import os, sys

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: Extract deep features for SUR prediction

Call pre-trained inceptionv3 for deep feature extraction

In [2]:
multigap_model = model_inception_multigap()

Instructions for updating:
Colocations handled automatically by placer.


Extract features from MCL-JCI database

In [3]:
df = pd.read_csv('csv_file/jnd_10_fold.csv')
feats_ref = []
feats_dist = []

for i in range(len(df)):
    im_ref_path = 'MCL_JCI_patches/reference_patches/' + df.iloc[i,1] + '.bmp'
    im_dist_path = 'MCL_JCI_patches/distorted_patches/' + df.iloc[i,2] + '.png'
    
    im_ref = Image.open(im_ref_path)
    im_ref = np.array(im_ref)/255.
    im_ref = np.expand_dims(im_ref, axis=0)
    feat_ref = multigap_model.predict(im_ref)
    feats_ref.append(feat_ref)
    
    im_dist = Image.open(im_dist_path)
    im_dist = np.array(im_dist)/255.
    im_dist = np.expand_dims(im_dist, axis=0)
    feat_dist = multigap_model.predict(im_dist)
    feats_dist.append(feat_dist)
    
    if i % 1000 == 0:
        print('%d images' % (i))
    

0 images
1000 images
2000 images
3000 images
4000 images
5000 images
6000 images
7000 images
8000 images
9000 images
10000 images
11000 images
12000 images
13000 images
14000 images
15000 images
16000 images
17000 images
18000 images
19000 images
20000 images
21000 images
22000 images
23000 images
24000 images


In [6]:
feats_ref = np.squeeze(np.array(feats_ref),axis=1)
feats_dist = np.squeeze(np.array(feats_dist),axis=1)
print(feats_ref.shape)
print(feats_dist.shape)

(25000, 10048)
(25000, 10048)


In [8]:
sur = df[['first_SUR']].values # SUR ground truth for first JND
print(sur.shape)

(25000, 1)


### Step 2: Train a shallow NN for SUR prediction

In [11]:
'''split for training, validation, and test'''
all_result = []

df = pd.read_csv('csv_file/jnd_10_fold.csv')
for i in range(10):
    if i is 9:
        test_set_id = i+1
        valid_set_id = 1
    else:
        valid_set_id = i+2
        test_set_id = i+1

    print('test_set_id:',test_set_id)
    print('valid_set_id:',valid_set_id)
    
    test_ids = df.loc[df.set==test_set_id]
    test_ids = test_ids.reset_index(drop=True)
    
    X_train_ref = feats_ref[(df.set!=valid_set_id)&(df.set!=test_set_id)]
    X_train_dist = feats_dist[(df.set!=valid_set_id)&(df.set!=test_set_id)]
    y_train = sur[(df.set!=valid_set_id)&(df.set!=test_set_id)]
    print(X_train_ref.shape)
    print(X_train_dist.shape)
    print(y_train.shape)


    X_valid_ref = feats_ref[df.set==valid_set_id]
    X_valid_dist = feats_dist[df.set==valid_set_id]
    y_valid = sur[df.set==valid_set_id]
    print(X_valid_ref.shape)
    print(X_valid_dist.shape)
    print(y_valid.shape)


    X_test_ref = feats_ref[df.set==test_set_id]
    X_test_dist = feats_dist[df.set==test_set_id]
    y_test = sur[df.set==test_set_id]
    print(X_test_ref.shape)
    print(X_test_dist.shape)
    print(y_test.shape)


    model = fc_model()
    model.compile(loss=keras.losses.mean_absolute_error,
                  optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0, amsgrad=False))

    # checkpoint
    filepath ='save_model/best_train.hdf5'
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                             monitor='val_loss', 
                                             verbose=0, 
                                             save_best_only=True, 
                                             mode='min')
    callbacks_list = [checkpoint]
    history = model.fit([X_train_ref,X_train_dist],y_train,
                        batch_size=128,
                        epochs=30,
                        verbose=0,
                        callbacks=callbacks_list,
                        validation_data=([X_valid_ref,X_valid_dist],y_valid))
    model.load_weights(filepath)

    y_pred = model.predict([X_test_ref,X_test_dist], batch_size=128)
    #print(y_pred)
    print(np.mean(np.absolute(y_pred-y_test)))
    K.clear_session()
    df_result = pd.concat([test_ids.iloc[:,0:3],pd.DataFrame(y_test,columns=['gt_SUR']),
                           pd.DataFrame(y_pred,columns=['pred_SUR'])],axis=1)
    all_result.append(df_result)

test_set_id: 1
valid_set_id: 2
(20000, 10048)
(20000, 10048)
(20000, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
0.052965000846106045
test_set_id: 2
valid_set_id: 3
(20000, 10048)
(20000, 10048)
(20000, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
0.06429984945606419
test_set_id: 3
valid_set_id: 4
(20000, 10048)
(20000, 10048)
(20000, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
0.05448957860550981
test_set_id: 4
valid_set_id: 5
(20000, 10048)
(20000, 10048)
(20000, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
0.06931097728023894
test_set_id: 5
valid_set_id: 6
(20000, 10048)
(20000, 10048)
(20000, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
(2500, 10048)
(2500, 10048)
(2500, 1)
0.04191579706849